<a href="https://colab.research.google.com/github/Edward-TL/La-Compra-Ideal/blob/master/VS_Buy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

#For the system
import os

#Manage of time
from datetime import datetime, timedelta
from pytz import timezone
import time
import re

#Manage of files
import pandas as pd
import csv

#scrap
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook
import requests

Mounted at /content/drive


# Data

In [4]:
from dataclasses import dataclass

@dataclass
class Page:
    name:str
    '''Page URl with name of the format replacers'''
    url : str      
    page_test : str
    '''Tuple with the name of the replacers'''
    url_replacers : tuple
    '''string that will replace the space of the users request on a search'''
    space_replacer : tuple
    '''All the next properties are tuples that contains:
    ("node: span", "attribute: class", "=predicate: text")'''
    boxes : tuple
    '''Products Info'''
    product_urls : tuple
    url_get : str
    # self.product_id = product_id
    name_and_images : tuple
    names_get : str
    images_get : str
    '''All the next properties are tuples that contains:
    ("node: span", "attribute: class", "=predicate: text")'''
    reviews : tuple
    stars : tuple
    price : tuple
    highlights : tuple
    '''dictionary'''
    money_dict : dict
    
    def adapt_url(self, Page, user_request, country_domain):
        check = str(type(Page.url_replacers))
        if country_domain[0] != ".":
            country_domain = '.' + country_domain
        
        
        if check != "<class 'str'>":
            adapted_url = Page.url.replace(Page.url_replacers[0], country_domain)
            for r in range(1, len(Page.url_replacers)):
                user_request_adapted = user_request.replace(' ', Page.space_replacer[r-1])
                adapted_url = adapted_url.replace(Page.url_replacers[r], user_request_adapted)
        else:
            user_request_adapted = user_request.replace(' ', Page.space_replacer[0])
            adapted_url = Page.url.replace(Page.url_replacers, user_request_adapted)
        
        return adapted_url

amazon_money_dict = {'mx' : {'coin' : '$',
                      'thousands': ',',
                      'decimal': '.',
                      'two_prices_sep' : ' - '}
}

Amazon = Page(name = 'Amazon',
    url='https://www.amazon.com{country}/s?k={user_request}',
    page_test='https://www.amazon.com.mx/s?k=audifonos+inalambricos',
    url_replacers=('{country}', '{user_request}'),
    space_replacer=['+'],
    boxes=('div', 'data-component-type', 's-search-result'),
    highlights=('div', 'class', 'a-row a-badge-region'),
    url_get='href',
    product_urls=('a', 'class', 'a-link-normal a-text-normal'),
    name_and_images=('div', 'class', 'a-section aok-relative s-image-square-aspect'),
    names_get='alt',
    images_get='src',
    reviews=('span', 'class', 'a-size-base'),
    stars=('span', 'class', 'a-icon-alt'),
    price=('span', 'class', 'a-offscreen'),
    money_dict=amazon_money_dict,
    #product_id('data-asin')
    )

ebay_money_dict = {'mx' : {'coin' : 'MXN $',
                      'thousands': ' ',
                      'decimal': '.',
                      'two_prices_sep' : ' a '}
            }

Ebay = Page(name='Ebay',
    url='https://www.ebay.com/sch/i.html?_nkw={user_request}',
    page_test='https://www.ebay.com/sch/i.html?_nkw=audifonos+inalambricos',
    url_replacers=('{user_request}'),
    space_replacer=['+'],
    boxes=('li', 'class', 's-item'),
    highlights=('span', 'class', 'ui-search-item__highlight-label__text'),
    product_urls=('div', 'class', 's-item__image'),
    url_get='href',
    name_and_images=('div', 'class', 's-item__image-wrapper'),
    names_get='alt',
    images_get='src',
    reviews=None,
    stars=None,
    price=('span', 'class', 's-item__price'),
    money_dict=ebay_money_dict,)

meli_money_dict = {'mx' : {'coin' : '$',
                      'thousands': ',',
                      'decimal': '.',
                      'two_prices_sep' : ' - '}
            }
            
Mercado_Libre = Page(name='Mercado Libre',
    url='https://listado.mercadolibre.com{country}/{user_request_1}#D[A:{user_request_2}]',
    page_test='https://listado.mercadolibre.com.mx/audifonos-inalambricos#D[A:audifonos%20inalambricos]',
    url_replacers=('{country}', '{user_request_1}', '{user_request_2}'),
    space_replacer=['-','%20'],
    boxes=('div', 'class', 'ui-search-result__wrapper'),
    highlights=('span', 'class', 'ui-search-item__highlight-label__text'),
    product_urls=('div', 'class', 'ui-search-result__image'),
    url_get='href',
    name_and_images=('div', 'class', 'slick-slide slick-active'),
    names_get='alt',
    images_get='data-src',
    reviews=None,
    stars=None,
    price=('span', 'class', 'price-tag ui-search-price__part'),
    money_dict=meli_money_dict,)

# wallmart_money_dict = {'mx' : {'coin' : '$',
#                       'thousands': ',',
#                       'decimal': '.',
#                       'two_prices_sep' : ' - '}
# }

# Wallmart = Page(name = 'Wallmart',
#     url='https://www.walmart.com{country}/productos?Ntt={user_request}',
#     url_replacers=('{country}', '{user_request}'),
#     space_replacer=['%20'],
#     boxes=('div', 'class', 'grid_product__30OQa'),
#     highlights=None,
#     product_urls=('a', 'class', 'nav-link_navLink__2oJ29 product_image__LcsAj'),
#     url_get='href',
#     name_and_images=('span', 'class', 'lazyload-wrapper'),
#     names_get='alt',
#     images_get='src',
#     reviews=None,
#     stars=None,
#     price=('p', 'data-automation-id', 'sale-price'),
#     money_dict=wallmart_money_dict,
#     #product_id('data-asin')
#     )

best_buy_money_dict = {'mx' : {'coin' : '$',
                      'thousands': ',',
                      'decimal': '.',
                      'two_prices_sep' : ' - '}
}

Best_Buy = Page(name = 'Best Buy',
    url='https://www.bestbuy.com{country}/c/buscar-best-buy/buscar?query={user_request}',
    page_test='https://www.bestbuy.com.mx/c/buscar-best-buy/buscar?query=audifonos+inalambricos',
    url_replacers=('{country}', '{user_request}'),
    space_replacer=['+'],
    boxes=('div', 'class', 'product-line-item-line'),
    highlights=None,
    product_urls=('div', 'class', 'product-title'),
    url_get='href',
    name_and_images=('div', 'class', 'col-xs-3 image-container'),
    names_get='alt',
    images_get='src',
    reviews=None,
    stars=None,
    price=('div', 'class', 'product-price'),
    money_dict=best_buy_money_dict,
    #product_id('data-asin')
    )


class headers:
    wallmart = {'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
                'accept-encoding':'gzip, deflate, br',
                'accept-language':'en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7',
                'cache-control':'max-age=0',
                'upgrade-insecure-requests':'1',
                'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'}

    h0 = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64;x64; rv:66.0) Gecko/20100101 Firefox/66.0",
            "Accept-Encoding":"gzip, deflate",
            "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "DNT":"1",
            "Connection":"close",
            "Upgrade-Insecure-Requests":"1"}

    h1 = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5)AppleWebKit/605.1.15 (KHTML, like Gecko)Version/12.1.1 Safari/605.1.15",
            "Accept-Encoding":"gzip, deflate",
            "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "DNT":"1",
            "Connection":"close",
            "Upgrade-Insecure-Requests":"1"}

    h2 = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36",
            "Accept-Encoding":"gzip, deflate",
            "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "DNT":"1",
            "Connection":"close",
            "Upgrade-Insecure-Requests":"1"}

    common = (h0, h2, h2)
    h3 = wallmart
    all_saved = (h0, h2, h2, wallmart)

# Scrape Funcs

In [5]:
#Web Manage
from bs4 import BeautifulSoup
import requests
import re

def extract_soup(url, header=0, just_status=False, just_soup=False):
    header = headers.all_saved[header]
    response = requests.get(url, headers=header)
    status = response.status_code

    soup = BeautifulSoup(response.text, 'lxml')

    if just_status==True:
        return status
    elif just_soup==True:
        return soup
    else:
        return soup, status

def search_boxes(soup, box_tuple, cents=False):
    #Search the kind of box specified by the touple
    if cents == True:
        boxes = soup.find(box_tuple[0], attrs={box_tuple[1] : box_tuple[2]})
    else:
        boxes = soup.find_all(box_tuple[0], attrs={box_tuple[1] : box_tuple[2]})

    return boxes

def get_brute_info(boxes_array, info_tuple):
    '''Returns the brute text from the html depending of the info you asked'''
    info = [None]*len(boxes_array)
    searcher = None
    i=0
    for box in boxes_array:
        searcher = search_boxes(box, info_tuple)
        info[i] = searcher
        searcher = None
        i += 1
    
    return info


# Data Filters

In [6]:
import re

def get_names(boxes_array, Page, test_all=False, test_len=False, position=None):
    names = [None]*len(boxes_array)
    name = Page.names_get

    if test_all == True:
            print(f'Boxes array len: {len(boxes_array)}')

    '''If you know want to know some info of an specific product by its position on the page.
    Like you know the position of the cheapest'''
    if position != None:
        searcher = search_boxes(boxes_array[position], Page.name_and_images)
        if searcher:
            if Page.name == 'Best Buy':
                image_name = searcher[0].img.get(name)
                name_split = image_name.split(' - ')
                names = name_split[1]
            else:
                names = searcher[0].img.get(name)
    else:
        '''For Testing the functions and Xpaths'''
        b=0
        if test_all == True:
            print('For Loop')
        for box in boxes_array:
            #Remember that boxes are arrays
            searcher = search_boxes(box, Page.name_and_images)
            if searcher:
                try:
                    if Page.name == 'Best Buy':
                        image_name = searcher[0].img.get(name)
                        name_split = image_name.split(' - ')
                        names[b] = name_split[1]
                    else:
                        names[b] = searcher[0].img.get(name)
                except:
                    name_message_error = f'''Value info:
                    box: {box}
                    Searcher: {searcher}
                    Searcher[0].img = {searcher[0].img}'''

                    raise ValueError(name_message_error)
                
            b +=1
    if test_all == True:
        print(names)
    
    elif test_len == True:
        print(len(names))

    return names


def get_images(boxes_array, Page, test_all=False, test_len=False, position=None):
    images = [None]*len(boxes_array)
    image = Page.images_get
    
    '''If you know want to know some info of an specific product by its position on the page.
    Like you know the position of the cheapest'''
    if position:
        searcher = search_boxes(boxes_array[position], Page.name_and_images)

        if searcher:
            images = searcher[0].img.get(image)
    else:
        '''For Testing the functions and Xpaths'''
        b=0
        for box in boxes_array:
            #Remember that boxes are arrays
            searcher = search_boxes(box, Page.name_and_images)
            if searcher:
                images[b] = searcher[0].img.get(image)
            b +=1

    if test_all == True:
        print(images)
    
    elif test_len == True:
        print('images:', len(images))

    return images

def get_products_urls(boxes_array, Page, test_all=False, test_len=False, position=None):
    urls = [None]*len(boxes_array)
    url = Page.url_get
    if test_all == True:
        print('Urls:')
    '''If you know want to know some info of an specific product by its position on the page.
    Like you know the position of the cheapest'''
    if position:
        searcher = search_boxes(boxes_array[position], Page.product_urls)
        if searcher:
            try:
                if Page.name == 'Amazon':
                    source_url = searcher[0].get(url)
                    position_url = 'https://www.amazon.com.mx' + source_url
                    
                else:
                    position_url = searcher[0].a.get(url)
                
                urls = position_url
            except:
                error_message = f'''Value info:
                Searcher: {searcher}
                searcher[0]: {searcher}
                searcher[0].a: searcher[0].a
                url: {Page.url_get}
                position: {position}
                '''

                raise ValueError(error_message)
    else:
        '''For Testing the functions and Xpaths'''
        b=0
        for box in boxes_array:
            if test_all == True:
                print('For loop:')
            #Remember that boxes are arrays
            searcher = search_boxes(box, Page.product_urls)
            if searcher:
                if test_all == True:
                    print(searcher[0].a)
                if Page.name == 'Amazon':
                    source_url = searcher[0].get(url)
                    urls[b] = 'https://www.amazon.com.mx' + source_url
                    
                else:
                    urls[b] = searcher[0].a.get(url)
            b +=1

    if test_all == True:
        print(urls)
    
    elif test_len == True:
        print('urls:', len(urls))
    return urls

def get_price(country, boxes_array, Page, test_all=False, test_len=False, position=None):
    price = [None]*len(boxes_array)
    coin_symbol = Page.money_dict[country]['coin']
    k_sep = Page.money_dict[country]['thousands']
    d_sep = Page.money_dict[country]['decimal']
    tps = Page.money_dict[country]['two_prices_sep']
    price_string = 'start'

    '''If you know want to know some info of an specific product by its position on the page.
    Like you know the position of the cheapest'''
    if position:
        searcher = search_boxes(boxes_array[position], Page.price)
        if searcher:
            try:
                price_string = searcher[0].get_text().split(tps)
                price_string = price_string[0].replace(coin_symbol,'').replace(k_sep,'').replace(d_sep,'.')
                #Special case
                price_string = re.findall(r'(\d+\.\d+)', price_string)
                price = float(price_string[0])
            except:
                error_message = f'''String index out of range. 
                Money dictionary: {Page.money_dict}
                Original String: {searcher[0].get_text()}
                Box #{position}'''
                raise ValueError(error_message)     
        
    #For Testing the functions and Xpaths
    else:
        b=0
        for box in boxes_array:
            #Remember that boxes are arrays
            searcher = search_boxes(box, Page.price)
            if test_all == True:
                print(searcher)
            if searcher:
                if country == 'mx':
                    try:
                        price_string = searcher[0].get_text().split(tps)
                        if test_all == True:
                            print(price_string)
                        price_string_bfre = price_string[0].replace(coin_symbol,'').replace(k_sep,'').replace(d_sep,'.')
                        if test_all == True:
                            print(price_string) 
                        #Ebays Special case
                        price_string_bfre = price_string_bfre.replace('\xa0','')
                        #Just in case
                        price_string_check = re.findall(rf"(\d+\.?\d+)", price_string_bfre)
                        price[b] = float(price_string_check[0])                        

                    except:
                        error_message = f'''Info about the Value. 
                        Money dictionary: {Page.money_dict}
                        Original String: {searcher[0].get_text()}
                        Before RegEx: {price_string_bfre}
                        Type Before RegEx: {type(price_string)}
                        Price string: {price_string}
                        Box #{b}'''
                        raise ValueError(error_message)                        
            b +=1

    if test_all == True:
        print('prices:', len(price), price)
    
    elif test_len == True:
        print('prices:', len(price))

    return price

def get_stars(boxes_array, Page, country='mx', test_all=False, test_len=False, position=None):
    stars = [None]*len(boxes_array)
    decimal_sep = Page.money_dict[country]['decimal']

    b=0
    for box in boxes_array:
        #Remember that boxes are arrays
        searcher = search_boxes(box, Page.stars)

        if searcher:
            if decimal_sep == '.':
                stars[b] = float(searcher[0].get_text()[:3])
            else:
                stars[b] = float(searcher[0].get_text()[:3].replace(decimal_sep,''))

        b +=1
    if test_all == True:
        print(stars)
    
    elif test_len == True:
        print('stars:', len(stars))
    return stars

def get_reviews(boxes_array, Page, country='mx', test_all=False, test_len=False, position=None):
    reviews = [None]*len(boxes_array)
    comma_sep = Page.money_dict[country]['thousands']

    b=0
    for box in boxes_array:
        #Remember that boxes are arrays
        searcher = search_boxes(box, Page.reviews)

        if searcher:
            if len(searcher) > 1:
                searcher = [searcher[0]]

            try:
                reviews[b] = int(searcher[0].get_text().replace(comma_sep,''))
            except:
                pass
        
        b +=1
    if test_all == True:
        print(reviews)
    
    elif test_len == True:
        print('reviews:', len(reviews))
    return reviews

def amazon_products_id(boxes_array, test=False):
    ids = [None]*len(boxes_array)

    b=0
    for box in boxes_array:
        #Remember that boxes are arrays
        if box:
            product_id = box.get('data-asin')
            ids[b] = 'www.amazon.com.mx/dp/' + product_id
            
        b +=1
    if test == True:
        print(ids)
    return ids

# Cheapest Functions

In [7]:
def ordered_dict(dictionary, reverse=True):
    import operator
    ordered_dict = dict(sorted(dictionary.items(), key=operator.itemgetter(1),reverse=True))

    return ordered_dict

def cheapest(array_prices, just_position=True, just_price=False, position_and_price=False, test=False):
    cheapest_position = 0
    cheapest_price = None
    
    for n in range(len(array_prices)):
        price = array_prices[n]
        if price:
            if cheapest_price == None:
                cheapest_price = price
            if test == True:
                print(f'number = {n} cheapest_price is {cheapest_price} and price check is {price}')

            if price < cheapest_price:
                if test == True:
                    print(f'price: {price} < {cheapest_price}')

                cheapest_price = price
                cheapest_position = n
        
    if position_and_price == True:
        just_position = False
        return cheapest_position, cheapest_price
    elif just_position == True:
        return cheapest_position
    elif just_price == True:
        return cheapest_price


def get_cheapest(cheapest_idx, products, cheapest_price=None, country=None, Page=None):
    products_type = str(type(products))
    if products_type == "<class 'bs4.element.ResultSet'>":
        
        if country and Page:
            cheapest_dict = {}
            cheapest_dict['name'] = get_names(products, Page, position=cheapest_idx)
            cheapest_dict['image'] = get_images(products, Page, position=cheapest_idx)
            cheapest_dict['url'] = get_products_urls(products, Page, position=cheapest_idx)
            cheapest_dict['price'] = cheapest_price
        else:
            if not country:
                raise ValueError("Missing country value")
            elif not Page:
                raise ValueError("Missing Page object")
            elif not cheapest_price:
                raise ValueError("Missing cheapest_price value (int)")
            else:
                raise ValueError(f'''Missing Page and country values.
            You can use this function with a dictionary with all the data preloaded as well''')
            
    elif products_type == "<class 'dict'>":
        cheapest_dict = {}
        products_dictionary = products
        for key in products_dictionary:
            cheapest_dict[key] = products_dictionary[key][cheapest_idx]
    else:
        error_in_product_type =  f"Products type must be a dict or a bs4.element.ResultSet with the boxes. Recived {type(products)}"
        raise ValueError(error_in_product_type)
    
    return cheapest_dict

# Main

In [8]:
def scrap_cheapest(user_request, Page, country='mx'):
    #Adapt the url
    request_url = Page.adapt_url(Page, user_request, country)
    #All the HTML of the page
    soup_request = extract_soup(request_url, 1, just_soup=True)
    # #HTML divided by products, and stored as elements of an array
    page_boxes = search_boxes(soup_request, Page.boxes)
    # From this part, could get better AFTER the 4 scrapers are made
    #From the Boxes, obtain the prices
    price_boxes = get_price(country, page_boxes, Page)
    #Obtain the cheapest from prices and then, you obtain the cheapest product as a dictionary
    cheapest_idx, cheapest_price = cheapest(price_boxes, position_and_price=True)
    cheapest_product_dictionary = get_cheapest(cheapest_idx, page_boxes, cheapest_price, country, Page)

    return cheapest_product_dictionary

In [9]:
user_request = 'audifonos inalambricos Marshall major III bluetooth'
pages = [Amazon, Mercado_Libre, Ebay, Best_Buy]
for page in pages:
    print(f'Searching {user_request} in -> {page.name}')
    cheapest_dict = scrap_cheapest(user_request, page)
    for key in cheapest_dict:
        print(key,':', cheapest_dict[key])
    print('\n')

Searching audifonos inalambricos Marshall major III bluetooth in -> Amazon
name : Jecoo Audífonos Bluetooth inalámbricos,Audífonos Impermerable con micrófono icorporado,portátil y táctil Mano Libre
image : https://m.media-amazon.com/images/I/61B5USDFewL._AC_UL320_.jpg
url : https://www.amazon.com.mx/gp/slredirect/picassoRedirect.html/ref=pa_sp_btf_aps_sr_pg1_1?ie=UTF8&adId=A0147565BOI6BINJFTSU&url=%2FJecoo-Aud%25C3%25ADfonos-inal%25C3%25A1mbricos-Impermerable-icorporado%2Fdp%2FB08DR3PK4D%2Fref%3Dsr_1_40_sspa%3Fdchild%3D1%26keywords%3Daud%25C3%25ADfonos%2Binal%25C3%25A1mbricos%2Bmarshall%2Bmajor%2Bii%2Bbluetooth%26qid%3D1601350514%26sr%3D8-40-spons%26psc%3D1&qualifier=1601350514&id=4484506651303780&widgetName=sp_btf
price : 399.0


Searching audifonos inalambricos Marshall major III bluetooth in -> Mercado Libre
name : Almohadillas Marshall Major Earpads Esponjas Marshal Mayor
image : https://http2.mlstatic.com/D_NQ_NP_603125-MLM40743739486_022020-V.webp
url : https://articulo.mercadoli

# Name match test

In [10]:
user_request = 'audifonos inalambricos Marshall Major III'
country = 'mx'
best_buy_url = Best_Buy.adapt_url(Best_Buy, user_request, country)

#All the HTML of the page
best_buy_soup = extract_soup(best_buy_url, 1, just_soup=True)


# #HTML divided by products, and stored as elements of an array
best_buy_boxes = search_boxes(best_buy_soup, Best_Buy.boxes)
# print(best_buy_boxes)

best_buy_products = {}

best_buy_products['names'] = get_names(best_buy_boxes, Best_Buy)
# #Best_Buy's images source (link)
best_buy_products['images'] = get_images(best_buy_boxes, Best_Buy)

best_buy_products['urls'] = get_products_urls(best_buy_boxes, Best_Buy)
best_buy_products['prices'] = get_price(country, best_buy_boxes, Best_Buy)

cheapest_idx = cheapest(best_buy_products['prices'])
cheapest_best_buy_product2 = get_cheapest(cheapest_idx, best_buy_products)

print(f'\nTest ONE:')
for key in cheapest_best_buy_product2:
    print(key, ':', cheapest_best_buy_product2[key])


Test ONE:
names : Audífonos On Ear Major III Bluetooth
images : https://pisces.bbystatic.com/image2/BestBuy_MX/images/products/1000/1000214586_sd.jpg;maxHeight=200;maxWidth=200
urls : https://www.bestbuy.com.mx/p/marshall-audifonos-on-ear-major-iii-bluetooth-negro-best-buy/1000214586
prices : 4149.0


In [54]:
import unicodedata
def standarize(word):
    word = word.lower()
    word = unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore')
    return word

def name_hits(user_request, product_names_array):
    request_splited = user_request.split()
    hits_dict = {}

    for product_name in product_names_array:
        points = 0
        product_splited = product_name.split()

        n = 0
        word = request_splited[n]
        for split in product_splited:
            split = standarize(split)
            while word != split and n < len(request_splited):
                word = standarize(request_splited[n])

                if word == split:
                    points += 1
                n += 1
            n = 0
            
        hits_dict[product_name] = points/len(product_name)
    return hits_dict

In [55]:
product_names_array = best_buy_products['names']
hits_dictionary = name_hits(user_request, product_names_array)
for key in hits_dictionary:
    print(key, ':', hits_dictionary[key])

Audífonos On Ear Major III Bluetooth : 0.08333333333333333
Audífonos On Ear Major III Bluetooth Voice Google : 0.061224489795918366
